## Zbiór danych

Dane pochodzą ze zbioru: https://www.apispreadsheets.com/datasets/129.

Przedstawiają one informacje meteorologiczne dotyczące pożarów lasu w 'Parque Natural de Montesinho' w Portugali.

## Preprocessing

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pandas_profiling import ProfileReport

In [ ]:
month_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
day_order = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
fires_data = pd.read_csv("forest_fires_dataset.csv")
fires_data['area_log'] = fires_data['area'].apply(np.log1p)
fires_data.columns

## Eksploracyjna Analiza Danych

In [ ]:
fires_data.head()

In [ ]:
fires_data.info()

In [ ]:
fires_data.describe()

In [ ]:
temp_data = fires_data.drop(["X", "Y", "area"], axis=1)
corre = temp_data.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corre, annot=True, square=True, fmt='.3f')
plt.title("Correlation heatmap of the data set")
plt.show()

In [ ]:
sns.histplot(data=fires_data, x="area_log")
plt.title("Countplot of area in logarithmic scale")
plt.xlabel("Area (log(x+1) scale)")
plt.show()

Zauważmy bardzo dużo wartości w pobliżu 0, postanowiłem sprawdzić ile ich jest równe 0.

In [ ]:
z = len(fires_data.loc[fires_data["area"] == 0])/len(fires_data["area"])
print(f"Area is equal to 0 in {z*100:.2f}% of rows")

Okazuje się że jest to prawie 50% danych. Pytanie czy dane te reprezentują brak pożaru, czy o powierzchni na tyle małej że nie została zmierzona. Jednak w pierwszym przypadku spodziewałbym się danych dość równo rozłożonych przez miesiące. Niestety nie jest to opisane na stronie zbioru danych. Szukając w internecie znalazłem dane że dla małych pożarów powierzchnia została ustalona na 0.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18, 9))
sns.countplot(data = fires_data, x = "month", order = month_order, ax=ax1, color = "tab:blue")
sns.barplot(x="month", y="area_log", order=month_order, data=fires_data, estimator=sum, ax=ax2, color = "tab:blue")
plt.suptitle("Count and sum of forest fires area per month", y=0.92)
ax1.set_xlabel("Month")
ax1.set_ylabel("Count of occurences")
ax2.set_xlabel("Month")
ax2.set_ylabel("Sum of area (log(x+1) scale)")
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18, 9))
plt.suptitle("Boxplots of area and temperature for the records", y=0.92)
sns.boxplot(x='month', y="area_log", data=fires_data, order=month_order, ax=ax1, color = "tab:blue")
sns.boxplot(x='month', y="temp", data=fires_data, order=month_order, ax=ax2, color = "tab:blue")
ax1.set_ylabel("Area in logarithmic scale")
ax2.set_ylabel("Temperature in Celsius")
plt.show()

Wygląda na to że istnieje powiązanie między temperaturą danego dnia a powierzchnią pożaru, jednak wydaje się być mniejsza niż bym się spodziewał.

In [ ]:
hue_var = "month"
y_variable = "area_log"
temp_data = fires_data.drop(["X", "Y", "area", "day"], axis = 1)
fst = sns.pairplot(temp_data, y_vars=y_variable, x_vars=temp_data.columns.values[:3], hue = hue_var, plot_kws={'alpha': 0.5}, height = 4)
scd = sns.pairplot(temp_data, y_vars=y_variable, x_vars=temp_data.columns.values[3:6], hue = hue_var, plot_kws={'alpha': 0.5}, height = 4)
trd = sns.pairplot(temp_data, y_vars=y_variable, x_vars=temp_data.columns.values[6:9], hue = hue_var, plot_kws={'alpha': 0.5}, height = 4)
fst.fig.suptitle("Distributions of variables in comparison with area and the months", y=1.0)
plt.show()

Jedyny klarowny rozdział miesięcy jest w wykresie area_log/DC. Według https://www.nwcg.gov/publications/pms437/cffdrs/fire-weather-index-system, DC to "The Drought Code" który wyznaczony jest na podstawie wysuszania się głębokich warstw gleby. Sensowne wydaje się powiązanie tego z miesiącami oraz z większymi pożarami.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(18, 9), sharex=True)
sns.countplot(data = fires_data, x = "day", order = day_order, ax=ax1, color = "tab:blue")
sns.barplot(x="day", y="area", order=day_order, data=fires_data, estimator=sum, ax=ax2, color = "tab:blue")
sns.barplot(x="day", y="area_log", order=day_order, data=fires_data, estimator=sum, ax=ax3, color = "tab:blue")
ax1.tick_params(
    axis='x',         
    which='both',      
    bottom=False,      
    labelbottom=False)
ax2.tick_params(
    axis='x',         
    which='both',      
    bottom=False,      
    labelbottom=False)
ax1.set(xlabel='')
ax2.set(xlabel='')
plt.suptitle("Count and sum of area and area in log(x+1) scale, by the day of the week", y = 0.92)
plt.show()

W weekend zdarza się najwięcej pożarów, tak samo pokrywają one największą powierzchnię. Zapewne związane jest to z uczęszczaniem społeczeństwa w wycieczkach do parku, które z oczywistych powodów wydarzają się w weekendy.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
sns.kdeplot(data = fires_data, x = "X", y = "Y", color="black", fill = True, cbar = True, ax = ax)
ax.invert_yaxis()
plt.title("Occurences of forest fires in the park area")
ticks = [1,2,3,4,5,6,7,8,9]
plt.xticks(ticks)
plt.yticks(ticks)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
sns.kdeplot(data = fires_data, x = "X", y = "Y", color="black", fill = True, weights = "area", cbar = True, ax = ax)
ax.invert_yaxis()
plt.title("Distribution of forests")
ticks = [1,2,3,4,5,6,7,8,9]
plt.xticks(ticks)
plt.yticks(ticks)
plt.show()

Mając te dwa wykresy oraz mape parku możemy sprawdzić które obszary doświadczają najwięcej i największe pożary. Mogą to być obszary najczęściej uczęszczane przez ludzi, najdalsze od zbiorników wodnych, itd. Brakuje nam danych by to dokładniej określić.

## Mapa Parku

Konwertowanie do pdf'a niestety usuwa mapę.

<img src="https://d3i71xaburhd42.cloudfront.net/0f529dc2b2b2bad22394454d4cba79e2c319f0b0/4-Figure2-1.png" width="700">

## Pandas-Profiling

In [ ]:
profile = ProfileReport(fires_data, title='Pandas Profiling Report', explorative=True)

In [ ]:
profile.to_notebook_iframe()

Narzędzia automatycznej eksploracji danych mogą być użyteczne, jednak jako dodatek lub wstęp do porządnej analizy danych. Oczywiście wszystko w nich wykonuje się automatycznie, więc nie ma miejsca na skupianie się na odpowiednich zmiennych, wiedzy o danej dziedzinie która jest znacząca dla analizy, itd. Może przyspieszyć prostą lecz żmudną część robienia histogramów, .info/.describe.